# TIPE 2023-2024 "Jeux et sports" 
### MORAIN Arthur MP2 Montaigne

## <ins>Optimisation de l'arrêt au stand en Formule 1</ins> 

### **<ins>III- Extraction de données :</ins>**   

#### <ins>a- Sources : </ins>

<ins>le module fastf1 :</ins>

https://docs.fastf1.dev/index.html

<ins>la base de données :</ins>

https://github.com/oracle-devrel/redbull-pit-strategy/tree/main

https://www.kaggle.com/datasets/jasperan/redbull-f1-pit-strategy

#### <ins>b- Ouverture des modules et de la base de données : </ins>

In [3]:
import pandas as pd
import os
import logging
import json
import pickle
import requests
import numpy as np
import fastf1
import matplotlib.pyplot as plt
import pickle
from fastf1.core import DataNotLoadedError

In [15]:
path = '/Users/morainarthur/Home/Datascience/redbull-pit-strategy/notebooks'
data_path = '/Users/morainarthur/Home/Datascience/redbull-pit-strategy/data'
cwd = os.getcwd()
 
# print the current directory
print("Current working directory is:", cwd)

Current working directory is: /Users/morainarthur/Home/Datascience/redbull-pit-strategy/notebooks


In [5]:
os.chdir(path)
fastf1.Cache.enable_cache(data_path)

#### <ins>c- Quelques fonctions pour travailler : </ins>

In [6]:
def get_lap_data(session, schedule, evnt, ses, EventDate):
    '''get lap data from a session and
       retun as a dataframe
    '''
    if len(session.laps)==0:
        return []
    lap = session.laps.to_dict()
    lap = pd.DataFrame.from_dict(lap)
    lap['RoundNumber'] = schedule['RoundNumber'][schedule['EventName']==evnt].values[0]
    lap['EventName'] = evnt
    lap['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    lap['session'] = ses
    lap['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return lap

def get_weather_data(session, schedule, evnt, ses,EventDate): 
    '''get weather data from a session and
       retun as a dataframe
    '''
    if len(session.weather_data)==0:
        return []
    weather = session.weather_data.to_dict()
    weather = pd.DataFrame.from_dict(weather)
    weather['RoundNumber'] = schedule['RoundNumber'][schedule['EventName']==evnt].values[0]
    weather['EventName'] = evnt
    weather['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    weather['session'] = ses
    weather['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return weather

def get_car_data(session, schedule, evnt, ses,EventDate):
    '''get car_data from a session and
       retun as a dataframe
    '''
    if len(session.car_data)==0:
        return []
    session.weather_data
    for ii in session.car_data:
        car_data = session.car_data[ii].to_dict()
        car_data = pd.DataFrame.from_dict(car_data)
        car_data['driver'] = ii
    car_data['RoundNumber'] = schedule['RoundNumber'][schedule['EventName']==evnt].values[0]
    car_data['EventName'] = evnt
    car_data['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    car_data['session'] = ses
    car_data['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return car_data

def get_position_data(session, schedule, evnt, ses, EventDate):
    '''get position_data from a session and
       retun as a dataframe
    '''    
    if len(session.pos_data)==0:
        return []
    for ii in session.pos_data:
        position = session.pos_data[ii].to_dict()
        position = pd.DataFrame.from_dict(position)
        position['driver'] = ii
    position['RoundNumber'] = schedule['RoundNumber'][schedule['EventName']==evnt].values[0]
    position['EventName'] = evnt
    position['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    position['session'] = ses
    position['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return position

def get_results(session, schedule, evnt, ses, EventDate):
    '''get results data from a session and
       retun as a dataframe
    '''
    if len(session.results)==0:
        return []
    result = session.results.to_dict()
    result = pd.DataFrame.from_dict(result).reset_index()
    result['RoundNumber'] = schedule['RoundNumber'][schedule['EventName']==evnt].values[0]
    result['EventName'] = evnt
    result['country'] = schedule['Country'][schedule['EventName']==evnt].values[0]
    result['session'] = ses
    result['EventDate'] = schedule[EventDate][schedule['EventName']==evnt].values[0]
    return result

#### <ins>d- Récupération des données télémétriques et météo : </ins>

In [5]:
# Pull data and store it in the data directory

sessionDateMap = {'Race': 'Session5Date',
          'Qualifying': 'Session4Date',
            'FP1': 'Session1Date',
            'FP2': 'Session2Date',
            'FP3': 'Session3Date',
            'S':  'Session4Date',
            'SQ':  'Session4Date'}

for year in [2018,2019, 2020, 2021, 2022]:
    laps = []
    weathers = []
    results = []
    
    sch = fastf1.get_event_schedule(year).to_dict()
    sch = pd.DataFrame.from_dict(sch)

        
    EventName = [s for s in sch['EventName'] if 'Grand' in s]
    
    for event in EventName:
        for session_type in ['FP1','FP2','FP3','S','SQ','Qualifying','Race']:
            
            eventDateColName = sessionDateMap[session_type]
            
            session = None
            
            try:
                session = fastf1.get_session(year, event, session_type) # call fastf1 to extract data
            except:
                print('Session: {} does not exist {}.'.format(session_type, event))
                session = None
            if session is not None:
                try:
                    session.load()
                except:
                    print('Session: ' + session_type + ' does not provide usable data {}.'.format(event))
                    session = None
            
            # if the session exists, extract all data with auxiliary functions from the above cells
            if session is not None:
                try:
                    ## get lap data for a session
                    laps.append(get_lap_data(session, sch, event, 
                                             session_type, eventDateColName))

                    ## get weather data for a session
                    weathers.append(get_weather_data(session, sch, event, 
                                                     session_type, eventDateColName))

                    ## get results for a session
                    results.append(get_results(session, sch, event, 
                                               session_type, eventDateColName))
                except DataNotLoadedError:
                    print('Trying to access data that isn\'t loaded yet')
                    continue


    ## save all extracted session data in a year           
    file = open('{}{}_{}.pkl'.format(data_path, 'laps', year), 'wb')
    pickle.dump(laps,file)
    file = open('{}{}_{}.pkl'.format(data_path, 'weathers', year), 'wb')
    pickle.dump(weathers,file)
    file = open('{}{}_{}.pkl'.format(data_path, 'results', year), 'wb')
    pickle.dump(results,file)

core           INFO 	Loading data for Australian Grand Prix - Practice 1 [v3.1.4]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            IN

Session: S does not exist Australian Grand Prix.
Session: SQ does not exist Australian Grand Prix.


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

Session: S does not exist Bahrain Grand Prix.
Session: SQ does not exist Bahrain Grand Prix.


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

...

req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

Session: FP3 does not exist São Paulo Grand Prix.


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

Session: S does not exist Abu Dhabi Grand Prix.
Session: SQ does not exist Abu Dhabi Grand Prix.


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to

#### <ins>e- Concaténation : </ins>

In [8]:
def concat_data_and_save(data_source, data_path):
    '''
    e.g. data_source is weather
    1. finds all pkl files in the data_path folder
    2. concat all
    3. saves the in 'data_source'.pickle
    '''
    data = []
    files = os.listdir(data_path)
    file = [x for x in files if x.endswith('.pkl') if data_source in x]
    
    for fl in file:
        print(fl)
        file = open('{}{}'.format(data_path, fl), 'rb')
        data.extend(pickle.load(file))
    data = pd.concat(data, axis=0)

    file = open('{}{}_data.pickle'.format(data_path, data_source), 'wb')
    pickle.dump(data,file)
    return

In [16]:
data_path = '/Users/morainarthur/Home/Datascience/redbull-pit-strategy/'
concat_data_and_save('weathers', data_path)
concat_data_and_save('laps', data_path)
concat_data_and_save('results', data_path)

dataweathers_2018.pkl
dataweathers_2019.pkl
dataweathers_2022.pkl
dataweathers_2021.pkl
dataweathers_2020.pkl
datalaps_2022.pkl
datalaps_2021.pkl
datalaps_2020.pkl
datalaps_2018.pkl
datalaps_2019.pkl
dataresults_2019.pkl
dataresults_2018.pkl
dataresults_2020.pkl
dataresults_2021.pkl
dataresults_2022.pkl


#### <ins>f- Remarque sur l'étape 3 : </ins>

-> Sur cette étape, je me suis clairement inspiré du travail trouvé avec la base de données, on a sûrement atteint le pic de difficulté 